I started by using Data Analyst GPT environment that is created by ChatGPT team.
First i tried uploading the csv directly since it is a publicly available data from GoogleBigQuery and asked chatGPT for a summary but uploading csv directly doesn't seem to work due to security issues and technical constraint. As a workaround chatGPT suggested to paste the data directly or to use external tools like jupyter notebook, since the data I have is too big for chatGPT, i decided to ask chatGPT for data analysis using jupyter notebook.

below is the suggested steps by chatGPT with my analysis of the results in bold text

## 1. Setup and Load Data

In [ ]:
# Install necessary libraries (if not already installed)
!pip install pandas matplotlib seaborn numpy

In [ ]:
# Import required libraries
import pandas as pd

# Load your CSV file
df = pd.read_csv('ecommerce-session-bigquery.csv') 

# View the first few rows of the dataset
df.head()

**viewing the first few rows is a basic and good way to start an analysis to check the general state of the data. We can see that the data in the city column have value such as "(not set)" and "not available in demo dataset" which would need to be cleaned before starting the analysis**

## 2. Explore the Dataset

In [ ]:
# Check the shape of the dataset (rows, columns)
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

# Display column names
print("Columns:", df.columns)

# Summary of the dataset (data types, null values, etc.)
df.info()

# Statistical summary for numerical columns
df.describe()

**another good way to be aware of the data in the dataset**

## 3. Check for Missing or Duplicated Data

In [ ]:
# Check for missing values
print("Missing Values:\n", df.isnull().sum())

# Drop rows with missing values (if necessary)
# df = df.dropna()

# Alternatively, fill missing values
# df['column_name'] = df['column_name'].fillna(0)  # Replace with appropriate value

# Check for duplicate rows
print("Duplicate Rows:", df.duplicated().sum())

# Drop duplicates if needed
# df = df.drop_duplicates()

**data quality check for missing or duplicate value, useful but doesn't take into account data that is not null but have weird value since chatGPT doesn't have access to the data (such as the city column value being "(not set)" or "not available in demo dataset")**

Afterwards ChatGPT suggested a few basic analysis but since we already have a case studies, i decided to directly ask ChatGPT on solutions all three problems while copy pasting the top 5 row of data so ChatGPT can get better context, here are the results:

# Problem 1: Identify Top Products Based on the Total Transaction Revenue Per Day

## Step 1: Inspect the Dataset

From the dataset snippet you shared earlier, the key columns we will use are:

date: Indicates the day of the session.  
v2ProductName: Name of the product.  
totalTransactionRevenue: Total revenue for the session (per user).  
productRevenue or itemRevenue: Revenue attributed to individual products (if available).


**since we already have totalTransactionRevenue, the productRevenue or itemRevenue is redundant so the key column is just going to be date, v2ProductName, and totalTransactionRevenue**

## Step 2: Data Preprocessing

In [ ]:
# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Ensure that revenue-related columns are numeric
df['totalTransactionRevenue'] = pd.to_numeric(df['totalTransactionRevenue'], errors='coerce')
df['productRevenue'] = pd.to_numeric(df['productRevenue'], errors='coerce')
df['itemRevenue'] = pd.to_numeric(df['itemRevenue'], errors='coerce')

# Fill missing revenue values with 0 (if they are NaN)
df['totalTransactionRevenue'] = df['totalTransactionRevenue'].fillna(0)
df['productRevenue'] = df['productRevenue'].fillna(0)
df['itemRevenue'] = df['itemRevenue'].fillna(0)

**this is a necessary step since all the value is currently in a string format**

## Step 3 & 4: Group Data by Product and Date & Identify Top Products Per Day

In [ ]:
# Group by 'date' and 'v2ProductName', summing up the revenue
daily_product_revenue = df.groupby(['date', 'v2ProductName'])['totalTransactionRevenue'].sum().reset_index()

# Sort the data by 'date' and 'totalTransactionRevenue' in descending order
daily_product_revenue = daily_product_revenue.sort_values(['date', 'totalTransactionRevenue'], ascending=[True, False])

# Get the top product per day
top_products_per_day = daily_product_revenue.groupby('date').head(1)  # Fetch the top product for each day

# Display the results
print(top_products_per_day)

**since this is a relatively simple questions, i think ChatGPT handles it quite well and gives satisfactory results, but the usage of v2ProductName instead of productSKU could cause a grouping mistake. Below i am checking the amount of v2ProductName per productSKU**

In [ ]:
# Count the number of unique v2ProductName per productSKU
unique_v2productnames = df.groupby('productSKU')['v2ProductName'].nunique().reset_index()

# Rename columns for clarity
unique_v2productnames.columns = ['productSKU', 'UniqueProductNames']

# Sort by the highest number of unique v2ProductName
unique_v2productnames = unique_v2productnames.sort_values('UniqueProductNames', ascending=False)

# Display the result
print(unique_v2productnames)

**Since we can see some product SKU have multiple product names, the results above might not be accurate. I would redo the same step but using the productSKU instead of v2ProductName for a more accurate results, and then reach out to the client on the proper productSKU to v2ProductName mapping.**

In [ ]:
# Group by 'date' and 'productSKU', summing up the revenue
daily_product_revenue = df.groupby(['date', 'productSKU'])['totalTransactionRevenue'].sum().reset_index()

# Sort the data by 'date' and 'totalTransactionRevenue' in descending order
daily_product_revenue = daily_product_revenue.sort_values(['date', 'totalTransactionRevenue'], ascending=[True, False])

# Get the top product per day
top_products_per_day = daily_product_revenue.groupby('date').head(1)  # Fetch the top product for each day

# Display the results
print(top_products_per_day)

**I would also add how often a product SKU become top transaction revenue where the total transaction revenue is not 0 to shows which product would be beneficial to focus on**

In [ ]:
top_products_per_day[top_products_per_day['totalTransactionRevenue'] > 0]['productSKU'].value_counts()

# Problem 2: Detect any anomalies, such as a sharp decrease or increase in the number of transactions for a specific product.

In [ ]:
# Aggregate transactions per product over time
transactions_per_product = df.groupby(['date', 'productSKU'])['transactions'].sum().reset_index()

# Fill missing or NaN transaction values with 0
transactions_per_product['transactions'] = transactions_per_product['transactions'].fillna(0)

# Sort by date for proper time-series analysis
transactions_per_product = transactions_per_product.sort_values(['productSKU', 'date'])

# Calculate the daily change in transactions for each product
transactions_per_product['daily_change'] = transactions_per_product.groupby('productSKU')['transactions'].diff()

# Add a column for the percentage change to better understand magnitude
transactions_per_product['percent_change'] = transactions_per_product.groupby('productSKU')['transactions'].pct_change() * 100

# Define thresholds for anomalies
threshold_percent_change = 50  # e.g., consider changes > 50% as anomalies
threshold_transaction_change = 10  # e.g., consider raw changes > 10 as anomalies

# Flag anomalies
transactions_per_product['anomaly'] = (
    (transactions_per_product['daily_change'].abs() > threshold_transaction_change) | 
    (transactions_per_product['percent_change'].abs() > threshold_percent_change)
)

# Filter only rows where anomalies are flagged
anomalies = transactions_per_product[transactions_per_product['anomaly']]

# Display anomalies
print(anomalies)


**The code above is the combination of several step from ChatGPT on how to detect anomalies, while the calculation is not wrong, we can see that the most of the results is -100% of inf, this is because there is not enough transaction per product and one transaction difference is enough to be detected as anomaly. below is a graph showing the total daily transaction by ChatGPT.**

In [ ]:
daily_transactions = df.groupby('date')['transactions'].sum().reset_index()

# Plot daily transactions as a line graph
plt.figure(figsize=(10, 6))
plt.plot(daily_transactions['date'], daily_transactions['transactions'], marker='o', linestyle='-', color='blue', label='Daily Transactions')

# Customize the plot
plt.title('Daily Transactions Over Time', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Transactions', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

# Show the plot
plt.show()

**Seeing as the highest total transaction in a day doesn't even reach 30 on later months of 2016 and there's barely any transactions on 2017, i would conclude that we don't have enough data points to conclude anomalies of daily product transactions**

# Problem 3: Identify the most profitable city or province based on the total transaction revenue. 

**Since we don't have province data, I'm going to use city data instead**

In [ ]:
# Group by city and sum the total transaction revenue
revenue_by_city = df.groupby('city')['totalTransactionRevenue'].sum().reset_index()

# Sort the cities by revenue in descending order
revenue_by_city = revenue_by_city.sort_values('totalTransactionRevenue', ascending=False)

# Display the top 10 most profitable cities
print("Top 10 Most Profitable Cities by Total Transaction Revenue:")
print(revenue_by_city.head(11))

**some of the city value is unavailable but I still prefer to include the value to highlight the amount of missing data to the client**

**Alternatively we can show the same data but uses Country data instead since the data is much cleaner**


In [ ]:
# Group by country and sum the total transaction revenue
revenue_by_country = df.groupby('country')['totalTransactionRevenue'].sum().reset_index()

# Sort the countries by revenue in descending order
revenue_by_country = revenue_by_country.sort_values('totalTransactionRevenue', ascending=False)

# Display the top 10 most profitable countries
print("Top 10 Most Profitable Countries by Total Transaction Revenue:")
print(revenue_by_country.head(10))

**Since the problem is relatively simple, the results from chatGPT could be used as is**